In [1]:
# Packages
import pandas as pd
import numpy as np
from collections import Counter
import nltk, spacy, re, json
from nltk.corpus import stopwords
from nltk.util import ngrams
nltk.download("punkt")
nltk.download('stopwords')

# Set up
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package punkt to /jet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /jet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
filename="../../data/twitter/elon_clean.csv"

In [3]:
elon = pd.read_csv(filename)

In [4]:
elon.head()

,username,date,retweets,favorites,text,mentions,hashtags,permalink
0,elonmusk,2019-04-02 20:38,993,9263,Dogecoin value may vary https://www. theonion.com/bitcoin-plunge -reveals-possible-vulnerabilities-in-craz-1821134169 …,NaN,NaN,https://twitter.com/elonmusk/status/1113178951403180032
1,elonmusk,2019-04-02 20:16,7238,55271,Dogecoin rulz pic.twitter.com/flWWUgAgLU,NaN,NaN,https://twitter.com/elonmusk/status/1113173498384441344
2,elonmusk,2019-04-02 19:40,368,9159,Uh oh,NaN,NaN,https://twitter.com/elonmusk/status/1113164389929160706
3,elonmusk,2019-04-02 09:24,1317,6176,Dogecoin might be my fav cryptocurrency. It’s pretty cool.,NaN,NaN,https://twitter.com/elonmusk/status/1113009339743100929
4,elonmusk,2019-04-02 09:21,1557,26925,Yup https:// twitter.com/nasa/status/11 12860196043452417 …,NaN,NaN,https://twitter.com/elonmusk/status/1113008497006804992


In [19]:
elon.shape

(6091, 8)

In [6]:
tweets=elon['text']

In [7]:
tweets

0       Dogecoin value may vary https://www. theonion.com/bitcoin-plunge -reveals-possible-vulnerabilities-in-craz-1821134169 …                             
1       Dogecoin rulz pic.twitter.com/flWWUgAgLU                                                                                                            
2       Uh oh                                                                                                                                               
3       Dogecoin might be my fav cryptocurrency. It’s pretty cool.                                                                                          
4       Yup https:// twitter.com/nasa/status/11 12860196043452417 …                                                                                         
5       Some challenges with ice formation in the cryogenic propellant prevalves. Hopefully overcome soon.                                                  
6       What could possibly go wrong?                     

In [8]:
nltk_twitter_tokens=[]
for tweet in tweets:
    nltk_twitter_tokens.append(nltk.casual_tokenize(tweet))

In [9]:
### Remember to follow the order here: match from first to last
regexes=(
    # Keep usernames together (any token starting with @, followed by A-Z, a-z, 0-9)
    r"(?:@[\w_]+)",
    # Keep hashtags together (any token starting with #, followed by A-Z, a-z, 0-9, _, or -)
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)",
    # Keep words with apostrophes, hyphens and underscores together
    r"(?:[a-z][a-z’'\-_]+[a-z])",
    # Keep all other sequences of A-Z, a-z, 0-9, _ together
    r"(?:[\w_]+)",
    # Everything else that's not whitespace
    r"(?:\S)")
big_regex="|".join(regexes)
extensible_tokenizer = re.compile(big_regex, re.VERBOSE | re.I | re.UNICODE)

def extensible_tokenize(text):
    return extensible_tokenizer.findall(text)

In [10]:
# Tokenize tweets data 
tweets_tokens=[]
for tweet in tweets:
    tokens=extensible_tokenize(tweet.lower())
    tweets_tokens.append(tokens)

In [11]:
# Stemming tweets
from nltk.stem import PorterStemmer
ps = PorterStemmer()

for tweets in tweets_tokens:
    for token in tweets:
        token=ps.stem(token)

In [12]:
tweets_tokens

[['dogecoin',
  'value',
  'may',
  'vary',
  'https',
  ':',
  '/',
  '/',
  'www',
  '.',
  'theonion',
  '.',
  'com',
  '/',
  'bitcoin-plunge',
  '-',
  'reveals-possible-vulnerabilities-in-craz',
  '-',
  '1821134169',
  '…'],
 ['dogecoin', 'rulz', 'pic', '.', 'twitter', '.', 'com', '/', 'flwwugaglu'],
 ['uh', 'oh'],
 ['dogecoin',
  'might',
  'be',
  'my',
  'fav',
  'cryptocurrency',
  '.',
  'it’s',
  'pretty',
  'cool',
  '.'],
 ['yup',
  'https',
  ':',
  '/',
  '/',
  'twitter',
  '.',
  'com',
  '/',
  'nasa',
  '/',
  'status',
  '/',
  '11',
  '12860196043452417',
  '…'],
 ['some',
  'challenges',
  'with',
  'ice',
  'formation',
  'in',
  'the',
  'cryogenic',
  'propellant',
  'prevalves',
  '.',
  'hopefully',
  'overcome',
  'soon',
  '.'],
 ['what', 'could', 'possibly', 'go', 'wrong', '?'],
 ['no'],
 ['you’re', 'so', 'right', '.', 'working', 'on', 'it', '!'],
 ['that',
  'car',
  'seemed',
  'so',
  'advanced',
  'when',
  'i',
  'watched',
  'that',
  'show',
  'a

In [17]:
tweets_tokens_clean=[]
for tweets in tweets_tokens:
    for token in tweets:
        tweets_tokens_clean.append(re.sub('[^a-zA-Z\-]','',token))

In [18]:
tweets_tokens_clean
#tweets_bag_of_words

['dogecoin',
 'value',
 'may',
 'vary',
 'https',
 '',
 '',
 '',
 'www',
 '',
 'theonion',
 '',
 'com',
 '',
 'bitcoin-plunge',
 '-',
 'reveals-possible-vulnerabilities-in-craz',
 '-',
 '',
 '',
 'dogecoin',
 'rulz',
 'pic',
 '',
 'twitter',
 '',
 'com',
 '',
 'flwwugaglu',
 'uh',
 'oh',
 'dogecoin',
 'might',
 'be',
 'my',
 'fav',
 'cryptocurrency',
 '',
 'its',
 'pretty',
 'cool',
 '',
 'yup',
 'https',
 '',
 '',
 '',
 'twitter',
 '',
 'com',
 '',
 'nasa',
 '',
 'status',
 '',
 '',
 '',
 '',
 'some',
 'challenges',
 'with',
 'ice',
 'formation',
 'in',
 'the',
 'cryogenic',
 'propellant',
 'prevalves',
 '',
 'hopefully',
 'overcome',
 'soon',
 '',
 'what',
 'could',
 'possibly',
 'go',
 'wrong',
 '',
 'no',
 'youre',
 'so',
 'right',
 '',
 'working',
 'on',
 'it',
 '',
 'that',
 'car',
 'seemed',
 'so',
 'advanced',
 'when',
 'i',
 'watched',
 'that',
 'show',
 'as',
 'a',
 'kid',
 '',
 'knight',
 'industries',
 'two',
 'thousand',
 'and',
 'even',
 'tinier',
 'ones',
 'for',
 'tardi

In [16]:
tweets_tokens_clean_hyphen=[]
for tweets in tweets_tokens_clean:
    print(tweets)
#     for token in tweets:
#         print(token)
        #tweets_tokens_clean_hyphen.append(re.sub('^\-','',token))

dogecoin
value
may
vary
https



www

theonion

com

bitcoin-plunge
-
reveals-possible-vulnerabilities-in-craz
-


dogecoin
rulz
pic

twitter

com

flwwugaglu
uh
oh
dogecoin
might
be
my
fav
cryptocurrency

its
pretty
cool

yup
https



twitter

com

nasa

status




some
challenges
with
ice
formation
in
the
cryogenic
propellant
prevalves

hopefully
overcome
soon

what
could
possibly
go
wrong

no
youre
so
right

working
on
it

that
car
seemed
so
advanced
when
i
watched
that
show
as
a
kid

knight
industries
two
thousand
and
even
tinier
ones
for
tardigrades
yes
going
forward

all
tesla
stores
will
be
tiny

have
tiny
cars
meant
to
say
young

but
spelled
it
wrong
ironic
stan
if
can
do
this

what
cant
they
do


solid
work
by
keen

as
usual
norway
https



twitter

com

elbilforeninge
n

status



yes
yes
seems
fair
some
people
use
their
hair
to
express
themselves

i
use
twitter

theonion
making
the
onion
articles
real

https



www

rollingstone

com

music

music-ne
ws

elon-musk-rap-song-r


as
well
as
test
rides
in
y
model
y

being
an
suv

is
about


bigger
than
model


so
will
cost
about


more

have
slightly
less
range
for
same
battery
model
y
unveil
event
on
march

at
la
design
studio
first
public
tesla
v


supercharger
station
goes
live
wed
pm





spacex
team
in
hawthorne
control

dragon
docked
to
station
above
pic

twitter

com

juwkorwjsh
spacex
crew
dragon
has
docked
with
international
spacestation
pic

twitter

com

cccsitllvj
yes

she
would
be
able
to
breathe
proximity
docking
sequence
underway

dragon
shadow
visible
on
on
spacestation
solar
arrays

pic

twitter

com

elsxglivf

one
day
we
will
be
there
to
see
for
ourselves
https



m

youtube

com

watch

v

gbxion
sjmky

f
something
like
this
could
work
everyone
knows
earth
is
hollow
i
am
not
a
dog

heres
a
pic
of
me

pic

twitter

com

huy
yrydnc
model
s
will
be
raised
to
mph
too
thank
you

may
you
have
good
luck
and
godspeed
with
your
spacecraft

thank
you
on
behalf
of
spacex

also

thank
you
to
nasa

witho

preburners

main
chamber
gaseous
ch

o

heavy
duty
spark
plugs

basically

a
of
insane
power
will
do

although
spacex
reddit
is
very
well-informed

really
impressive
analysis
by
some
commenters

engine
use
methox
torch
igniters

green
tinge
is
either
camera
saturation
or
a
tiny
bit
of
copper
from
the
chamber

pic

twitter

com

yfhnx
fiod
pic

twitter

com

chegbjtgkz
first
firing
of
starship
raptor
flight
engine

so
proud
of
great
work
by
spacex
team


pic

twitter

com

satjihs
we
will
when
it
rolls
out
yes
at
spacex
texas
with
engineering
team
getting
ready
to
fire
new
raptor
rocket
engine
pic

twitter

com

acfm
atyw
soon
only
a
matter
of
time
pic

twitter

com

eqnhk
that
might
be
the
ultimate
superpower
how
model

performs
on
a
long
road
trip
https



insideevs

com

lists

how-tesl
a-model
-

-
cracked-code-ev-road-trip


im
openly
moderate

there

i
said
it

haha
how
is
this
even
real

no

payload
will
actually
increase
exactly
i
do
like
nonsense
memes

its
true

yes
pic

twitt

that
speed

it
will
feel
like
teleporting
within
a
city

yes
this
can
be
applied
to
any
autonomous

electric
vehicle

not
just
tesla
tesla
in
boringcompany
tunnel
with
retractable
wheel
gear
that
turns
a
car
into
a
rail-guided
train

back
again
pic

twitter

com

ainosmi
boring
product
launch
will
be
streamed
live
at
pm
california
time
tonight
http



boringcompany

com
boring
is
developing
an
entirely
new
system
of
transport
same
applies
to
those
who
didnt
order
pup
exactly

its
a
good
way
to
go

get
people
with
more
money
to
pay
for
the
development

yeah

i
wrote
that
back
in


almost

years
ago

clayton
is
wrong

new
tech
is
always
expensive

tech
disruption
occurs
at

high
end


eg
computers

cell
phones

it
takes
many
design
iterations

vast
economies
of
scale
to
achieve
mass
market
affordability

pic

twitter

com

zvflyaz
honored
to
work
with
amazing
teams
at
spacex

tesla

https



twitter

com

teslarati

stat
us



totally
false

buried
in
this
incredibly
misleading
wsj
artic

to
spacex
tech
tree
build

falcon

second
stage
will
be
upgraded
to
be
like
a
mini-bfr
ship
but
comments
are
la

hawthorne
demonstration
tunnel
activates
dec


runs
parallel
to

freeway
from
crenshaw
blvd
to


i
think
twitter
is
deleting
fake

scam
or
maybe
even
inactive
accounts

my
follower
count
decreased
by

k
over
the
past
few
days

pic

twitter

com

gtpxlomzn
sure

we
can
do
it
zeroth
law
foundation
of
psychohistory
i
am
not
true
b
or

b
love
oumuamua
jokes
remember
the
future
oh
hi
guys

lol
https



www-m

cnn

com







hea
lth

oumuamua-alien-probe-harvard-intl

index

html

r

https

a

f

fwww

google

com

f

yes

germany
has
outstanding
engineering
talent
just
a
few
mins

which
is
mostly
accelerating

decelerating
pic

twitter

com

xgj
bchpl
so
underappreciated
ok
ok
walked
full
length
of
boring
co
tunnel
under
la
tonight

disturbingly
long

on
track
for
opening
party
dec


will
be
very
one-dimensional

sounds
like
pure
luxury

when
i
said

sponge


i
was
actually
spea

...







spacex-soft-lands-falcon
-

-
rocket-first-stage


looks
like
body
slam

maybe
from
a
self-generated
wave
detailed
review
of
rocket
telemetry
needed
to
tell
if
due
to
initial
splashdown
or
subsequent
tip
over
and
body
slam
rocket
booster
reentry

landing
burn

leg
deploy
were
good

but
lost
hull
integrity
right
after
splashdown

aka
kaboom

flight

of
falcon

was
good

all
six
orbcomm
satellites
deployed
on
target

leroytwin
teslacanada
a
lot
of
canadian
superchargers
are
in
work
and
will
start
going
live
in
the
next
month
or
so
tesla
superchargers
delivered
over
gwh
last
month

long
distance
driving
growing
exponentially
http



www

teslamotors

com

blog

superchar
ging-milestone

tesla
model
s
named
top
model
in
perceived
quality
http



usat

ly

litde
via
usatoday
test
flight
of
falcon

-
r
with
deployable
grid
fins
for
better
hypersonic
thru
subsonic
control
http



m

youtube

com

watch

v

dglbid
vgem

feature

youtu

be

goal
is
for
unsubsidized
solar
power
to
cost
les

we
now
have
all
the
pieces
of
the
puzzle
to
bring
the
rocket
back
home

rocket
booster
relit
twice

supersonic
retro

landing


but
spun
up
due
to
aero
torque

so
fuel
centrifuged

we
flamed
out
launch
was
good

all
satellites
deployed
at
the
targeted
orbit
insertion
vectors

pic

twitter

com

suymh
wph
you
can
watch
the
launch
at
http



www

spacex

com

webcast

pic

twitter

com

uveugrug
looks
good
for
a
launch
attempt
today

upper
winds
slightly
exceed
loads
in
high
subsonic
regime

but
improving

water
ice
on
mars
http



gizmodo

com

nasas-curiosit
y-rover-just-found-water-in-martian-soil
-


sensible
piece
on
status
of
climate
change
by
badastronomer
http



www

slate

com

blogs

badastr
onomy







climatechangedenialjamesdelingpoletellsitlikeitisnt

html

falcon

launch
window
is
sunday
and
monday

assuming
good
weather
at
vandenberg
air
force
base
good
progress
harnessing
that
big
fusion
reactor
in
the
sky
http



www

businessinsider

com

scientists-cre
ate-world-rec

at
disposal
to
help
in
any
way

a
whitehouse
petition
to
allow
direct
sales
of
cars
by
manufacturers

https



petitions

whitehouse

gov

petition

overt
urn-franchise-laws-limit-auto-manufacturers-selling-their-vehicles-directly-consumers

rlshblzr

please
vote
here
too

if
you
live
in
texas
and
are
pro
tesla

please
vote
in
the
myabj
opinion
poll

http



www

bizjournals

com

austin

poll

re
sults



interesting



putin
unveils

b
drive
for
russian
space
supremacy
http



phys

org

news


-

-
p
utin-unveils-bn-russian-space

html


via
physorgcom

batasrki
stay
tuned
for
a
big
supercharger
announcement
in
a
few
weeks

definitely
have
you
covered

jbsiegelmd
thanks

hearing
from
constituents
is
super
important
to
counteract
the
influence
of
the
dealer
lobby

new
york
judge
states




dealers
cannot
utilize
the
franchised
dealer
act
as
a
means
to
sue
their
competitors


exactly


new
york
judge
just
ruled
in
favor
tesla

dismissing
the
legal
attack
by
auto
dealers
to
prevent
dir

congrats
to
dmetcalf
for
being
the
first
to
drive
a
tesla
model
s
over

miles
on
a
single
charge

interesting
possible
answer
to
the
fermi
paradox
http



news

discovery

com

space

do-robot
s-rule-the-galaxy
-


html

am
happy
to
report
that
tesla
was
narrowly
cash
flow
positive
last
week

continued
improvement
expected
through
year
end

uranium
ore
now
available
on
amazon

so
much
more
convenient
than
uzbekistan
awesomexmasgifts
http



www

amazon

com

gp

product

b
xxm

ref

ohdetailso
si

shervin
thanks
shervin

cant
put
my
finger
on
it

but
for
some
reason
the
newsstand
is
looking
particularly
good
right
now
pic

twitter

com

grarj
xy
but
if
humanity
wishes
to
become
a
multi-planet
species

then
we
must
figure
out
how
to
move
millions
of
people
to
mars

and

yes

i
do
in
fact
know
that
this
sounds
crazy

that
is
not
lost
on
me

nor
i
do
think
spacex
will
do
this
alone

millions
of
people
needed
for
mars
colony

so
k

would
just
be
the
number
moving
to
mars
per
year
http



n

time

all
systems
green

dragonlaunch
launch
video
of
falcon

from
cape
canaveral
tracking
cameras
http



www

youtube

com

watch

v

vkqbf
vomm

feature

youtubegdataplayer

dragonlaunch
navigation
bay
pointing
to
deep
space
and
star
map
being
generated
by
star
tracker
one

yes

dragonlaunch
dragon
spaceship
opens
the
navigation
pod
bay
door
without
hesitation

so
much
nicer
than
hal



dragonlaunch
also
big
thanks
to
the
air
force

faa
and
all
of
our
partners
for
their
support
of
this
mission

huge
appreciation
for
nasa

without
whom
we
could
not
even
have
started

let
alone
reached
this
far

falcon
flew
perfectly


dragon
in
orbit

comm
locked
and
solar
arrays
active


feels
like
a
giant
weight
just
came
off
my
back


t
minus

seconds

terminal
autosequence
initiated

dragonlaunch
t
minus

minutes



entering
terminal
count
dragonlaunch
major
tesla
milestone

all
crash
testing
is
complete
for



max

safety
rating

cars
can
now
be
built
for
sale
to
public

just
did
an
interview
wi

In [172]:
tweets_tokens_clean_hyphen

['dogecoin',
 'value',
 'may',
 'vary',
 'https',
 ':',
 '/',
 '/',
 'www',
 '.',
 'theonion',
 '.',
 'com',
 '/',
 'bitcoin-plunge',
 '',
 'reveals-possible-vulnerabilities-in-craz',
 '',
 '1821134169',
 '…',
 'dogecoin',
 'rulz',
 'pic',
 '.',
 'twitter',
 '.',
 'com',
 '/',
 'flwwugaglu',
 'uh',
 'oh',
 'dogecoin',
 'might',
 'be',
 'my',
 'fav',
 'cryptocurrency',
 '.',
 'it’s',
 'pretty',
 'cool',
 '.',
 'yup',
 'https',
 ':',
 '/',
 '/',
 'twitter',
 '.',
 'com',
 '/',
 'nasa',
 '/',
 'status',
 '/',
 '11',
 '12860196043452417',
 '…',
 'some',
 'challenges',
 'with',
 'ice',
 'formation',
 'in',
 'the',
 'cryogenic',
 'propellant',
 'prevalves',
 '.',
 'hopefully',
 'overcome',
 'soon',
 '.',
 'what',
 'could',
 'possibly',
 'go',
 'wrong',
 '?',
 'no',
 'you’re',
 'so',
 'right',
 '.',
 'working',
 'on',
 'it',
 '!',
 'that',
 'car',
 'seemed',
 'so',
 'advanced',
 'when',
 'i',
 'watched',
 'that',
 'show',
 'as',
 'a',
 'kid',
 '!',
 'knight',
 'industries',
 'two',
 'thousand

In [166]:
tweets_tokens_clean_space=[]
for tweets in tweets_tokens_clean_hyphen:
    for token in tweets:
        tweets_tokens_clean_space.append(token)

In [167]:
tweets_tokens_clean_space

['dogecoin',
 'value',
 'may',
 'vary',
 'https',
 ':',
 '/',
 '/',
 'www',
 '.',
 'theonion',
 '.',
 'com',
 '/',
 'bitcoin-plunge',
 '-',
 'reveals-possible-vulnerabilities-in-craz',
 '-',
 '1821134169',
 '…',
 'dogecoin',
 'rulz',
 'pic',
 '.',
 'twitter',
 '.',
 'com',
 '/',
 'flwwugaglu',
 'uh',
 'oh',
 'dogecoin',
 'might',
 'be',
 'my',
 'fav',
 'cryptocurrency',
 '.',
 'it’s',
 'pretty',
 'cool',
 '.',
 'yup',
 'https',
 ':',
 '/',
 '/',
 'twitter',
 '.',
 'com',
 '/',
 'nasa',
 '/',
 'status',
 '/',
 '11',
 '12860196043452417',
 '…',
 'some',
 'challenges',
 'with',
 'ice',
 'formation',
 'in',
 'the',
 'cryogenic',
 'propellant',
 'prevalves',
 '.',
 'hopefully',
 'overcome',
 'soon',
 '.',
 'what',
 'could',
 'possibly',
 'go',
 'wrong',
 '?',
 'no',
 'you’re',
 'so',
 'right',
 '.',
 'working',
 'on',
 'it',
 '!',
 'that',
 'car',
 'seemed',
 'so',
 'advanced',
 'when',
 'i',
 'watched',
 'that',
 'show',
 'as',
 'a',
 'kid',
 '!',
 'knight',
 'industries',
 'two',
 'thousa

## Extract Features

In [138]:
# remove punctuation
tweets_tokens_flat=[]
for tokens in tweets_tokens:
    for token in tokens:
        tweets_tokens_flat.append(token)

In [139]:
tweets_tokens_flat

['dogecoin',
 'value',
 'may',
 'vary',
 'https',
 ':',
 '/',
 '/',
 'www',
 '.',
 'theonion',
 '.',
 'com',
 '/',
 'bitcoin-plunge',
 '-',
 'reveals-possible-vulnerabilities-in-craz',
 '-',
 '1821134169',
 '…',
 'dogecoin',
 'rulz',
 'pic',
 '.',
 'twitter',
 '.',
 'com',
 '/',
 'flwwugaglu',
 'uh',
 'oh',
 'dogecoin',
 'might',
 'be',
 'my',
 'fav',
 'cryptocurrency',
 '.',
 'it’s',
 'pretty',
 'cool',
 '.',
 'yup',
 'https',
 ':',
 '/',
 '/',
 'twitter',
 '.',
 'com',
 '/',
 'nasa',
 '/',
 'status',
 '/',
 '11',
 '12860196043452417',
 '…',
 'some',
 'challenges',
 'with',
 'ice',
 'formation',
 'in',
 'the',
 'cryogenic',
 'propellant',
 'prevalves',
 '.',
 'hopefully',
 'overcome',
 'soon',
 '.',
 'what',
 'could',
 'possibly',
 'go',
 'wrong',
 '?',
 'no',
 'you’re',
 'so',
 'right',
 '.',
 'working',
 'on',
 'it',
 '!',
 'that',
 'car',
 'seemed',
 'so',
 'advanced',
 'when',
 'i',
 'watched',
 'that',
 'show',
 'as',
 'a',
 'kid',
 '!',
 'knight',
 'industries',
 'two',
 'thousa

In [140]:
# # find most frequent words
# Counter(tweets_tokens_flat).most_common(100)

Sentiment Analysis

In [141]:
tweets_tokens_flat

['dogecoin',
 'value',
 'may',
 'vary',
 'https',
 ':',
 '/',
 '/',
 'www',
 '.',
 'theonion',
 '.',
 'com',
 '/',
 'bitcoin-plunge',
 '-',
 'reveals-possible-vulnerabilities-in-craz',
 '-',
 '1821134169',
 '…',
 'dogecoin',
 'rulz',
 'pic',
 '.',
 'twitter',
 '.',
 'com',
 '/',
 'flwwugaglu',
 'uh',
 'oh',
 'dogecoin',
 'might',
 'be',
 'my',
 'fav',
 'cryptocurrency',
 '.',
 'it’s',
 'pretty',
 'cool',
 '.',
 'yup',
 'https',
 ':',
 '/',
 '/',
 'twitter',
 '.',
 'com',
 '/',
 'nasa',
 '/',
 'status',
 '/',
 '11',
 '12860196043452417',
 '…',
 'some',
 'challenges',
 'with',
 'ice',
 'formation',
 'in',
 'the',
 'cryogenic',
 'propellant',
 'prevalves',
 '.',
 'hopefully',
 'overcome',
 'soon',
 '.',
 'what',
 'could',
 'possibly',
 'go',
 'wrong',
 '?',
 'no',
 'you’re',
 'so',
 'right',
 '.',
 'working',
 'on',
 'it',
 '!',
 'that',
 'car',
 'seemed',
 'so',
 'advanced',
 'when',
 'i',
 'watched',
 'that',
 'show',
 'as',
 'a',
 'kid',
 '!',
 'knight',
 'industries',
 'two',
 'thousa

In [142]:
# remove punctuation
tweets_tokens_flat=[re.sub('[^a-zA-Z\-]' ,'',w) for w in tweets_tokens_flat] 
# remove non-word hyphen
tweets_tokens_flat=[re.sub('^\-' ,'',w) for w in tweets_tokens_flat] 
# remove empty tokens
tweets_tokens_flat=[w for w in tweets_tokens_flat if w]#

In [143]:
tweets_tokens_flat

['dogecoin',
 'value',
 'may',
 'vary',
 'https',
 'www',
 'theonion',
 'com',
 'bitcoin-plunge',
 'reveals-possible-vulnerabilities-in-craz',
 'dogecoin',
 'rulz',
 'pic',
 'twitter',
 'com',
 'flwwugaglu',
 'uh',
 'oh',
 'dogecoin',
 'might',
 'be',
 'my',
 'fav',
 'cryptocurrency',
 'its',
 'pretty',
 'cool',
 'yup',
 'https',
 'twitter',
 'com',
 'nasa',
 'status',
 'some',
 'challenges',
 'with',
 'ice',
 'formation',
 'in',
 'the',
 'cryogenic',
 'propellant',
 'prevalves',
 'hopefully',
 'overcome',
 'soon',
 'what',
 'could',
 'possibly',
 'go',
 'wrong',
 'no',
 'youre',
 'so',
 'right',
 'working',
 'on',
 'it',
 'that',
 'car',
 'seemed',
 'so',
 'advanced',
 'when',
 'i',
 'watched',
 'that',
 'show',
 'as',
 'a',
 'kid',
 'knight',
 'industries',
 'two',
 'thousand',
 'and',
 'even',
 'tinier',
 'ones',
 'for',
 'tardigrades',
 'yes',
 'going',
 'forward',
 'all',
 'tesla',
 'stores',
 'will',
 'be',
 'tiny',
 'have',
 'tiny',
 'cars',
 'meant',
 'to',
 'say',
 'young',
 '

In [144]:
# remove stopwords
tweets_tokens_flat = [w for w in tweets_tokens_flat if w not in stopwords.words("english")]

In [145]:
tweets_tokens_flat

['dogecoin',
 'value',
 'may',
 'vary',
 'https',
 'www',
 'theonion',
 'com',
 'bitcoin-plunge',
 'reveals-possible-vulnerabilities-in-craz',
 'dogecoin',
 'rulz',
 'pic',
 'twitter',
 'com',
 'flwwugaglu',
 'uh',
 'oh',
 'dogecoin',
 'might',
 'fav',
 'cryptocurrency',
 'pretty',
 'cool',
 'yup',
 'https',
 'twitter',
 'com',
 'nasa',
 'status',
 'challenges',
 'ice',
 'formation',
 'cryogenic',
 'propellant',
 'prevalves',
 'hopefully',
 'overcome',
 'soon',
 'could',
 'possibly',
 'go',
 'wrong',
 'youre',
 'right',
 'working',
 'car',
 'seemed',
 'advanced',
 'watched',
 'show',
 'kid',
 'knight',
 'industries',
 'two',
 'thousand',
 'even',
 'tinier',
 'ones',
 'tardigrades',
 'yes',
 'going',
 'forward',
 'tesla',
 'stores',
 'tiny',
 'tiny',
 'cars',
 'meant',
 'say',
 'young',
 'spelled',
 'wrong',
 'ironic',
 'stan',
 'cant',
 'solid',
 'work',
 'keen',
 'usual',
 'norway',
 'https',
 'twitter',
 'com',
 'elbilforeninge',
 'n',
 'status',
 'yes',
 'yes',
 'seems',
 'fair',
 '

In [146]:
# find most frequent words
Counter(tweets_tokens_flat).most_common(100)

[('com', 1210),
 ('tesla', 717),
 ('twitter', 643),
 ('https', 587),
 ('http', 430),
 ('pic', 376),
 ('www', 345),
 ('model', 333),
 ('yes', 290),
 ('good', 289),
 ('like', 250),
 ('spacex', 232),
 ('car', 229),
 ('v', 193),
 ('would', 192),
 ('rocket', 191),
 ('next', 181),
 ('great', 165),
 ('time', 162),
 ('people', 160),
 ('one', 159),
 ('cars', 156),
 ('falcon', 151),
 ('dont', 148),
 ('us', 144),
 ('launch', 144),
 ('first', 144),
 ('right', 135),
 ('status', 134),
 ('also', 134),
 ('thanks', 133),
 ('need', 130),
 ('new', 129),
 ('soon', 127),
 ('much', 127),
 ('make', 125),
 ('watch', 125),
 ('get', 123),
 ('year', 123),
 ('better', 122),
 ('x', 121),
 ('even', 120),
 ('high', 119),
 ('yeah', 119),
 ('youtube', 117),
 ('love', 117),
 ('really', 115),
 ('way', 114),
 ('thats', 113),
 ('true', 112),
 ('p', 112),
 ('production', 111),
 ('work', 107),
 ('maybe', 106),
 ('actually', 106),
 ('coming', 104),
 ('team', 101),
 ('exactly', 100),
 ('im', 97),
 ('instagram', 97),
 ('dragon

In [147]:
self_stopwords=['com','twitter','https','http','www','v','x','p','would']

In [148]:
tweets_tokens_flat=[w for w in tweets_tokens_flat if w not in self_stopwords]

In [149]:
Counter(tweets_tokens_flat).most_common(100)

[('tesla', 717),
 ('pic', 376),
 ('model', 333),
 ('yes', 290),
 ('good', 289),
 ('like', 250),
 ('spacex', 232),
 ('car', 229),
 ('rocket', 191),
 ('next', 181),
 ('great', 165),
 ('time', 162),
 ('people', 160),
 ('one', 159),
 ('cars', 156),
 ('falcon', 151),
 ('dont', 148),
 ('us', 144),
 ('launch', 144),
 ('first', 144),
 ('right', 135),
 ('status', 134),
 ('also', 134),
 ('thanks', 133),
 ('need', 130),
 ('new', 129),
 ('soon', 127),
 ('much', 127),
 ('make', 125),
 ('watch', 125),
 ('get', 123),
 ('year', 123),
 ('better', 122),
 ('even', 120),
 ('high', 119),
 ('yeah', 119),
 ('youtube', 117),
 ('love', 117),
 ('really', 115),
 ('way', 114),
 ('thats', 113),
 ('true', 112),
 ('production', 111),
 ('work', 107),
 ('maybe', 106),
 ('actually', 106),
 ('coming', 104),
 ('team', 101),
 ('exactly', 100),
 ('im', 97),
 ('instagram', 97),
 ('dragon', 97),
 ('think', 94),
 ('many', 91),
 ('week', 91),
 ('space', 91),
 ('going', 90),
 ('go', 89),
 ('test', 88),
 ('sure', 87),
 ('back', 

In [ ]:
# sentiment analysis
# bi-gram
# dictionary based on eda

In [118]:
#tweets_tokens_text=' '.join(tweets_tokens_flat)